In [60]:
from os import path
import io
import os

import pandas as pd
from pydub import AudioSegment
import requests
from tqdm.auto import tqdm
from nltk.translate.bleu_score import sentence_bleu

# Чтение файлов

In [57]:
df_valid = pd.read_csv('../data/valid.csv')
df_train = pd.read_csv('../data/train.csv')

# Yandex Cloud STT

In [4]:
# brew install ffmpeg

import sys
sys.path.append('/path/to/ffmpeg')

In [55]:
with open('../secrets/yandex-traslate-apikey.txt', 'r') as file:
    apikey = file.read().replace('\n', '')
    
def yandex_stt(filename: str, lang:str = 'ru-RU', sampleRateHertz: int = 8000) -> str:

    fn = f'../data/speech/{filename}'
    if os.path.getsize(fn) < 500:
        return ''
    
    url = f"https://stt.api.cloud.yandex.net/speech/v1/stt:recognize?lang={lang}&format=lpcm&sampleRateHertz={sampleRateHertz}"

    sound = AudioSegment.from_mp3(fn)
    sound = sound.set_frame_rate(sampleRateHertz)
    buffer = io.BytesIO()
    sound.export(buffer, format='wav')
    files = {'file': buffer.read()[:1_000_000]}

    headers = {
        'Authorization': f'Api-Key {apikey}',
    }
    response = requests.request("POST", url, headers=headers, files=files)
    return response.json()['result']


In [59]:
tqdm.pandas()

def apply_fn(speech_fn):
    return yandex_stt(speech_fn)

df_valid['yandex_tts'] = df_valid['speech_fn'].progress_apply(apply_fn)
df_valid.to_csv('../data/yandex_tts_valid.csv')

100%|██████████| 200/200 [02:50<00:00,  1.17it/s]


In [73]:
df_valid.apply(lambda row: sentence_bleu([row['text_ru']],row['yandex_tts']), axis=1).mean()


0.6407131801825868